# 🔬 Single Well Simulation Test

This notebook focuses on simulating **one specific well** to:
- Understand decline curve behavior
- Validate simulation accuracy
- Debug any issues with individual well production

## Contents
1. [Setup & Data Loading](#1.-Setup-&-Data-Loading)
2. [Select a Well](#2.-Select-a-Well)
3. [Analyze Well Production History](#3.-Analyze-Well-Production-History)
4. [Run Single Well Simulation](#4.-Run-Single-Well-Simulation)
5. [Compare Results](#5.-Compare-Results)

---
## 1. Setup & Data Loading

In [ ]:
# Standard imports
import sys
import math
from pathlib import Path
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional

# Data analysis
import polars as pl
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Set up paths
SIMULATION_ROOT = Path(r"D:\Mu\Simulation")
DATA_DIR = SIMULATION_ROOT / "data"
sys.path.insert(0, str(SIMULATION_ROOT))

# Configure display
pl.Config.set_tbl_rows(50)
pd.set_option('display.max_columns', 50)

# Matplotlib style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

print("✅ Imports loaded")

In [ ]:
# Load data
print("📊 Loading data...")
fact = pl.read_csv(DATA_DIR / "houston_fact_combined_cleaned.csv")
dim = pl.read_csv(DATA_DIR / "qi_di_b_factor_dimension.csv")

print(f"✅ Fact: {fact.shape[0]:,} rows")
print(f"✅ Dim: {dim.shape[0]:,} rows")

# Add date column
fact = fact.with_columns([
    pl.col('production_vol_date').str.to_date().alias('prod_date')
])

In [ ]:
# Import simulation modules
try:
    from simulation import SimulationRunner, BasinModel, WellAgent
    from config.simulation_config import (
        WellType, DeclineType, DriveMechanism, WellConfiguration,
        WELL_TYPE_PARAMETERS, CONSTANTS,
    )
    print("✅ Simulation modules loaded!")
except ImportError as e:
    print(f"❌ Import error: {e}")

---
## 2. Select a Well

Choose a specific well to simulate. Options:
- Pick a high-producing well for clear trends
- Pick a specific API number you're interested in

In [ ]:
# Get summary of all wells
well_summary = (
    fact
    .group_by('api')
    .agg([
        pl.col('prod_date').min().alias('first_date'),
        pl.col('prod_date').max().alias('last_date'),
        pl.col('bopd').max().alias('max_bopd'),
        pl.col('bopd').mean().alias('avg_bopd'),
        pl.col('bopd').sum().alias('total_oil'),
        pl.col('play_type').first().alias('play_type'),
        pl.col('basin').first().alias('basin'),
        pl.col('production_status').last().alias('status'),
        pl.len().alias('days_of_data'),
    ])
    .sort('total_oil', descending=True)
)

print(f"📊 Total wells available: {well_summary.shape[0]:,}")
print("\n🔝 Top 20 wells by total production:")
well_summary.head(20)

In [ ]:
# ========================================================
# 🎯 SELECT YOUR WELL HERE
# ========================================================
# Option 1: Use the top producer
SELECTED_API = well_summary.row(0, named=True)['api']

# Option 2: Specify a specific API number
# SELECTED_API = 42015000750000  # Uncomment and change to your API

# Get well info
well_info = well_summary.filter(pl.col('api') == SELECTED_API).row(0, named=True)

print(f"\n🎯 SELECTED WELL:")
print("=" * 60)
print(f"  API Number:     {SELECTED_API}")
print(f"  Well Type:      {well_info['play_type']}")
print(f"  Basin:          {well_info['basin']}")
print(f"  Status:         {well_info['status']}")
print(f"  Date Range:     {well_info['first_date']} to {well_info['last_date']}")
print(f"  Days of Data:   {well_info['days_of_data']:,}")
print(f"  Max Production: {well_info['max_bopd']:.1f} bopd")
print(f"  Avg Production: {well_info['avg_bopd']:.1f} bopd")
print(f"  Total Oil:      {well_info['total_oil']:,.0f} bbl")

---
## 3. Analyze Well Production History

In [ ]:
# Get all production data for selected well
well_production = (
    fact
    .filter(pl.col('api') == SELECTED_API)
    .sort('prod_date')
)

print(f"📊 Production records for API {SELECTED_API}:")
print(f"   Total records: {well_production.shape[0]}")
print(f"\n📋 First 10 production records:")
well_production.head(10)

In [ ]:
# Plot production history
prod_df = well_production.to_pandas()

fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# 1. Daily production over time
ax1 = axes[0, 0]
ax1.plot(prod_df['prod_date'], prod_df['bopd'], 'b-', linewidth=1, alpha=0.7)
ax1.scatter(prod_df['prod_date'], prod_df['bopd'], s=5, alpha=0.3)
ax1.set_xlabel('Date')
ax1.set_ylabel('Production (bopd)')
ax1.set_title(f'Daily Production - API {SELECTED_API}', fontsize=14, fontweight='bold')
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
ax1.xaxis.set_major_locator(mdates.MonthLocator(interval=6))
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)
ax1.grid(True, alpha=0.3)

# 2. Cumulative production
ax2 = axes[0, 1]
prod_df['cumulative'] = prod_df['bopd'].cumsum()
ax2.fill_between(prod_df['prod_date'], prod_df['cumulative'] / 1000, alpha=0.3, color='green')
ax2.plot(prod_df['prod_date'], prod_df['cumulative'] / 1000, 'g-', linewidth=2)
ax2.set_xlabel('Date')
ax2.set_ylabel('Cumulative Production (thousand bbl)')
ax2.set_title('Cumulative Production', fontsize=14, fontweight='bold')
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
ax2.xaxis.set_major_locator(mdates.MonthLocator(interval=6))
plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45)
ax2.grid(True, alpha=0.3)

# 3. Production distribution
ax3 = axes[1, 0]
ax3.hist(prod_df['bopd'], bins=50, color='orange', alpha=0.7, edgecolor='black', linewidth=0.5)
ax3.axvline(prod_df['bopd'].mean(), color='red', linestyle='--', linewidth=2, label=f"Mean: {prod_df['bopd'].mean():.1f}")
ax3.axvline(prod_df['bopd'].median(), color='blue', linestyle='--', linewidth=2, label=f"Median: {prod_df['bopd'].median():.1f}")
ax3.set_xlabel('Daily Production (bopd)')
ax3.set_ylabel('Frequency')
ax3.set_title('Production Distribution', fontsize=14, fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Monthly production
ax4 = axes[1, 1]
monthly = prod_df.set_index('prod_date')['bopd'].resample('M').sum()
ax4.bar(monthly.index, monthly.values / 1000, width=20, color='purple', alpha=0.7)
ax4.set_xlabel('Month')
ax4.set_ylabel('Monthly Production (thousand bbl)')
ax4.set_title('Monthly Production', fontsize=14, fontweight='bold')
ax4.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
ax4.xaxis.set_major_locator(mdates.MonthLocator(interval=6))
plt.setp(ax4.xaxis.get_majorticklabels(), rotation=45)
ax4.grid(True, alpha=0.3)

plt.suptitle(f'Well Analysis: API {SELECTED_API} ({well_info["play_type"]})', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Print statistics
print(f"\n📊 Production Statistics:")
print(f"   Min:    {prod_df['bopd'].min():.1f} bopd")
print(f"   Max:    {prod_df['bopd'].max():.1f} bopd")
print(f"   Mean:   {prod_df['bopd'].mean():.1f} bopd")
print(f"   Median: {prod_df['bopd'].median():.1f} bopd")
print(f"   Std:    {prod_df['bopd'].std():.1f} bopd")
print(f"   Total:  {prod_df['bopd'].sum():,.0f} bbl")

In [ ]:
# Get decline parameters from dimension table for this well type
well_type_upper = well_info['play_type'].upper()

dim_params = (
    dim
    .filter(
        (pl.col('well_type') == well_type_upper) &
        (pl.col('model') == 'hyperbolic')
    )
)

print(f"📊 Decline parameters for {well_type_upper} (Hyperbolic):")
if dim_params.shape[0] > 0:
    avg_qi = dim_params['Qi'].mean()
    avg_di = dim_params['Di'].mean()
    avg_b = dim_params['b_factor'].mean()
    print(f"   Avg Qi: {avg_qi:.1f} bopd")
    print(f"   Avg Di: {avg_di:.4f} (annual decline rate)")
    print(f"   Avg b:  {avg_b:.2f}")
    dim_params.head(10)
else:
    print("   No matching parameters found in dimension table")
    avg_qi = well_info['max_bopd']
    avg_di = 0.3
    avg_b = 0.5

---
## 4. Run Single Well Simulation

In [ ]:
# Map play type to simulation well type
def map_play_type(play_type: str) -> str:
    mapping = {
        'CONVENTIONAL ONSHORE': 'Conventional Onshore',
        'CONVENTIONAL OFFSHORE': 'Conventional Offshore',
        'SHALE': 'Shale Oil',
        'TIGHT': 'Tight Oil',
        'CONVENTIONAL TIGHT': 'Conventional Tight',
    }
    return mapping.get(play_type.upper(), 'Conventional Onshore')

# Prepare well configuration
WELL_CONFIG = {
    'api_number': SELECTED_API,
    'well_type': map_play_type(well_info['play_type']),
    'initial_production': float(well_info['max_bopd']),  # Use max as initial
    'b_factor': float(avg_b) if avg_b else 0.5,
    'nominal_decline_rate': float(avg_di) if avg_di else 0.3,
    'decline_type': 'Hyperbolic',
    'location': well_info['basin'],
    'basin': well_info['basin'],
    'production_start_date': datetime(well_info['first_date'].year, well_info['first_date'].month, well_info['first_date'].day),
}

print("📋 Well Configuration for Simulation:")
print("=" * 60)
for key, value in WELL_CONFIG.items():
    print(f"   {key:25}: {value}")

In [ ]:
# Define simulation period (match the data range)
SIM_START = WELL_CONFIG['production_start_date']
SIM_END = datetime(well_info['last_date'].year, well_info['last_date'].month, well_info['last_date'].day)

print(f"📅 Simulation Period:")
print(f"   Start: {SIM_START.date()}")
print(f"   End:   {SIM_END.date()}")
print(f"   Days:  {(SIM_END - SIM_START).days}")

In [ ]:
# Create simulation runner
try:
    runner = SimulationRunner(
        total_recoverable_resources=1_000_000_000,  # Large enough for single well
        start_date=SIM_START,
        end_date=SIM_END,
        basin_name=well_info['basin'],
        market_center='Houston',
        country='US',
    )
    print("✅ SimulationRunner created")
    print(f"   Total steps: {runner.total_steps}")
except Exception as e:
    print(f"❌ Error: {e}")
    runner = None

In [ ]:
# Add the single well
if runner:
    try:
        well_state = runner.add_well(**WELL_CONFIG)
        print(f"✅ Well added successfully!")
        print(f"   ID: {well_state['id']}")
        print(f"   Type: {well_state['well_type']}")
        print(f"   Initial Production: {well_state['initial_production']:.1f} bopd")
        print(f"   EUR: {well_state['eur']:,.0f} bbl")
    except Exception as e:
        print(f"❌ Error adding well: {e}")

In [ ]:
# Run the simulation
if runner:
    print("🚀 Running single well simulation...")
    
    sim_results = []
    runner.start()
    step = 0
    
    while runner.model.simulation_active:
        state = runner.step()
        step += 1
        
        sim_state = state['state']
        wells = sim_state['basin']['wells']
        
        if wells:
            well = wells[0]  # Single well
            sim_results.append({
                'date': sim_state['current_date'],
                'daily_production': well['daily_production'],
                'potential_production': well['daily_potential_production'],
                'cumulative': well['cumulative_production'],
                'days_on_prod': well['days_on_production'],
                'is_active': well['is_active'],
            })
        
        # Progress
        if step % 100 == 0:
            print(f"   Day {step}: {sim_results[-1]['daily_production']:.1f} bopd")
    
    print(f"\n✅ Simulation complete! {step} days")
    
    # Convert to DataFrame
    sim_df = pd.DataFrame(sim_results)
    sim_df['date'] = pd.to_datetime(sim_df['date'])
    sim_df.set_index('date', inplace=True)
    
    print(f"\n📊 Simulation Results:")
    print(f"   Start prod: {sim_df['daily_production'].iloc[0]:.1f} bopd")
    print(f"   End prod:   {sim_df['daily_production'].iloc[-1]:.1f} bopd")
    print(f"   Total:      {sim_df['daily_production'].sum():,.0f} bbl")

---
## 5. Compare Results

In [ ]:
# Prepare actual production for comparison
actual_df = prod_df.copy()
actual_df['date'] = pd.to_datetime(actual_df['prod_date'])
actual_df = actual_df.set_index('date')[['bopd']]
actual_df.columns = ['actual']

# Merge simulated and actual
comparison = sim_df[['daily_production']].join(actual_df, how='outer')
comparison.columns = ['Simulated', 'Actual']
comparison = comparison.dropna(how='all').fillna(0)

print(f"📊 Comparison data: {len(comparison)} days")
comparison.head(20)

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Daily production comparison
ax1 = axes[0, 0]
ax1.plot(comparison.index, comparison['Actual'], label='Actual', 
         color='#2ecc71', linewidth=1.5, alpha=0.8)
ax1.plot(comparison.index, comparison['Simulated'], label='Simulated', 
         color='#e74c3c', linewidth=2, linestyle='--')
ax1.set_xlabel('Date')
ax1.set_ylabel('Production (bopd)')
ax1.set_title(f'Simulated vs Actual - API {SELECTED_API}', fontsize=14, fontweight='bold')
ax1.legend(fontsize=12)
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
ax1.xaxis.set_major_locator(mdates.MonthLocator(interval=6))
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)
ax1.grid(True, alpha=0.3)

# 2. Cumulative comparison
ax2 = axes[0, 1]
sim_cum = comparison['Simulated'].cumsum()
actual_cum = comparison['Actual'].cumsum()
ax2.plot(comparison.index, actual_cum / 1000, label='Actual', color='#2ecc71', linewidth=2)
ax2.plot(comparison.index, sim_cum / 1000, label='Simulated', color='#e74c3c', linewidth=2, linestyle='--')
ax2.set_xlabel('Date')
ax2.set_ylabel('Cumulative (thousand bbl)')
ax2.set_title('Cumulative Production', fontsize=14, fontweight='bold')
ax2.legend(fontsize=12)
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
ax2.xaxis.set_major_locator(mdates.MonthLocator(interval=6))
plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45)
ax2.grid(True, alpha=0.3)

# 3. Error
ax3 = axes[1, 0]
error = comparison['Simulated'] - comparison['Actual']
ax3.fill_between(comparison.index, error, where=error >= 0, color='#e74c3c', alpha=0.5, label='Over-estimate')
ax3.fill_between(comparison.index, error, where=error < 0, color='#2ecc71', alpha=0.5, label='Under-estimate')
ax3.axhline(0, color='black', linewidth=0.5)
ax3.set_xlabel('Date')
ax3.set_ylabel('Difference (bopd)')
ax3.set_title('Simulation Error (Simulated - Actual)', fontsize=14, fontweight='bold')
ax3.legend()
ax3.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
ax3.xaxis.set_major_locator(mdates.MonthLocator(interval=6))
plt.setp(ax3.xaxis.get_majorticklabels(), rotation=45)
ax3.grid(True, alpha=0.3)

# 4. Scatter correlation
ax4 = axes[1, 1]
ax4.scatter(comparison['Actual'], comparison['Simulated'], alpha=0.3, s=10, c='#3498db')
max_val = max(comparison['Actual'].max(), comparison['Simulated'].max()) * 1.1
ax4.plot([0, max_val], [0, max_val], 'r--', linewidth=2, label='Perfect Fit')
ax4.set_xlabel('Actual (bopd)')
ax4.set_ylabel('Simulated (bopd)')
ax4.set_title('Correlation Plot', fontsize=14, fontweight='bold')
ax4.set_xlim(0, max_val)
ax4.set_ylim(0, max_val)
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.suptitle(f'Single Well Simulation Results - {well_info["play_type"]}', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Print comparison statistics
print("\n📊 COMPARISON STATISTICS:")
print("=" * 60)
print(f"\n🔵 ACTUAL Production:")
print(f"   Total:      {comparison['Actual'].sum():,.0f} bbl")
print(f"   Average:    {comparison['Actual'].mean():.1f} bopd")
print(f"   Max:        {comparison['Actual'].max():.1f} bopd")
print(f"   Start:      {comparison['Actual'].iloc[0]:.1f} bopd")
print(f"   End:        {comparison['Actual'].iloc[-1]:.1f} bopd")

print(f"\n🔴 SIMULATED Production:")
print(f"   Total:      {comparison['Simulated'].sum():,.0f} bbl")
print(f"   Average:    {comparison['Simulated'].mean():.1f} bopd")
print(f"   Max:        {comparison['Simulated'].max():.1f} bopd")
print(f"   Start:      {comparison['Simulated'].iloc[0]:.1f} bopd")
print(f"   End:        {comparison['Simulated'].iloc[-1]:.1f} bopd")

print(f"\n📏 ERROR METRICS:")
total_error = comparison['Simulated'].sum() - comparison['Actual'].sum()
pct_error = total_error / comparison['Actual'].sum() * 100 if comparison['Actual'].sum() > 0 else 0
print(f"   Total Error:  {total_error:,.0f} bbl ({pct_error:+.1f}%)")

# MAPE
mask = comparison['Actual'] > 0
if mask.sum() > 0:
    mape = (abs(comparison['Simulated'][mask] - comparison['Actual'][mask]) / comparison['Actual'][mask]).mean() * 100
    print(f"   MAPE:         {mape:.1f}%")

# Correlation
corr = comparison['Simulated'].corr(comparison['Actual'])
print(f"   Correlation:  {corr:.4f}")

# RMSE
rmse = np.sqrt(((comparison['Simulated'] - comparison['Actual']) ** 2).mean())
print(f"   RMSE:         {rmse:.1f} bopd")

In [ ]:
# Monthly comparison
monthly_sim = comparison['Simulated'].resample('M').sum()
monthly_actual = comparison['Actual'].resample('M').sum()

monthly_compare = pd.DataFrame({
    'Simulated': monthly_sim / 1000,
    'Actual': monthly_actual / 1000,
    'Error': (monthly_sim - monthly_actual) / 1000,
    'Error_Pct': ((monthly_sim - monthly_actual) / monthly_actual * 100).replace([np.inf, -np.inf], 0)
})

print("\n📊 Monthly Comparison (thousand bbl):")
monthly_compare

In [ ]:
# Experiment with different parameters
print("\n🔧 PARAMETER SENSITIVITY TEST")
print("=" * 60)
print(f"Current parameters:")
print(f"   Qi = {WELL_CONFIG['initial_production']:.1f} bopd")
print(f"   Di = {WELL_CONFIG['nominal_decline_rate']:.4f}")
print(f"   b  = {WELL_CONFIG['b_factor']:.2f}")
print(f"\nTo tune the model, try adjusting:")
print(f"   - Increase b: Slower decline (harmonic-like)")
print(f"   - Decrease b: Faster decline (exponential-like)")
print(f"   - Increase Di: Steeper initial decline")
print(f"   - Decrease Di: Slower decline rate")

---
## 📝 Summary

This notebook tested a single well simulation for validation purposes.

### Key Findings:
- See the comparison statistics above for accuracy metrics
- The decline curve parameters (Qi, Di, b) can be tuned for better fit
- Real production often has variability not captured by smooth decline curves

### Next Steps:
1. Try different b-factor values to improve fit
2. Consider using well-specific parameters instead of averages
3. Test wells from different play types